In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**This version uses 1 dim values for distance. Weights are not shared.**

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras import optimizers
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Dropout, Add, merge, Subtract
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import concatenate, Activation
from keras.layers import Lambda
from keras import backend as K
from keras.utils import np_utils
import h5py
import time
import keras.layers

Using TensorFlow backend.


In [ ]:
def mat_mul(A, B):
    return tf.matmul(A, B)


def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    id = f['id'][:]
    return (data, label, id)


def rotate_point_cloud(batch_data):
    """ Randomly rotate the point clouds to augument the dataset
        rotation is per shape based along up direction
        Input:
          BxNx3 array, original batch of point clouds
        Return:
          BxNx3 array, rotated batch of point clouds
    """
    rotated_data = np.zeros(batch_data.shape, dtype=np.float32)
    for k in range(batch_data.shape[0]):
        rotation_angle = np.random.uniform() * 2 * np.pi
        cosval = np.cos(rotation_angle)
        sinval = np.sin(rotation_angle)
        rotation_matrix = np.array([[cosval, 0, sinval],
                                    [0, 1, 0],
                                    [-sinval, 0, cosval]])
        shape_pc = batch_data[k, ...]
        rotated_data[k, ...] = np.dot(shape_pc.reshape((-1, 3)), rotation_matrix)
    return rotated_data


def jitter_point_cloud(batch_data, sigma=0.01, clip=0.05):
    """ Randomly jitter points. jittering is per point.
        Input:
          BxNx3 array, original batch of point clouds
        Return:
          BxNx3 array, jittered batch of point clouds
    """
    B, N, C = batch_data.shape
    assert(clip > 0)
    jittered_data = np.clip(sigma * np.random.randn(B, N, C), -1 * clip, clip)
    jittered_data += batch_data
    return jittered_data
  

# number of points in each sample
num_points = 2048

# define optimizer
adam = optimizers.Adam(lr=0.001, decay=0.7)

def base_model(input):
  # input_Transformation_net
  input_points = input
  x = Convolution1D(64, 1, activation='relu', input_shape=(num_points, 3))(input_points)
  x = BatchNormalization()(x)
  x = Convolution1D(128, 1, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Convolution1D(1024, 1, activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPooling1D(pool_size=num_points)(x)
  x = Dense(512, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dense(256, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
  input_T = Reshape((3, 3))(x)

  # forward net
  g = Lambda(mat_mul, arguments={'B': input_T})(input_points)
  g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
  g = BatchNormalization()(g)
  g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
  g = BatchNormalization()(g)

  # feature transform net
  f = Convolution1D(64, 1, activation='relu')(g)
  f = BatchNormalization()(f)
  f = Convolution1D(128, 1, activation='relu')(f)
  f = BatchNormalization()(f)
  f = Convolution1D(1024, 1, activation='relu')(f)
  f = BatchNormalization()(f)
  f = MaxPooling1D(pool_size=num_points)(f)
  f = Dense(512, activation='relu')(f)
  f = BatchNormalization()(f)
  f = Dense(256, activation='relu')(f)
  f = BatchNormalization()(f)
  f = Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
  feature_T = Reshape((64, 64))(f)

  # forward net
  g = Lambda(mat_mul, arguments={'B': feature_T})(g)
  g = Convolution1D(64, 1, activation='relu')(g)
  g = BatchNormalization()(g)
  g = Convolution1D(128, 1, activation='relu')(g)
  g = BatchNormalization()(g)
  g = Convolution1D(20, 1, activation='relu')(g)
  g = BatchNormalization()(g)

  # global_feature
  global_feature = MaxPooling1D(pool_size=num_points)(g)
  out = Flatten()(global_feature)
  
  return out

input_points1=Input(shape=(num_points, 3))
input_points2=Input(shape=(num_points, 3))
input_points3=Input(shape=(num_points, 3))

out1 = base_model(input=input_points1)
out2 = base_model(input=input_points2)
out3 = base_model(input=input_points3)


# ------------------------------------- Concatenate ---

# Getting the L1 Distance between the 2 encodings
L1_layer = Lambda(lambda tensor:tf.reduce_sum(K.square(tensor[0] - tensor[1]), axis=1, keepdims=True))
# L1_layer = Lambda(lambda tensor:K.sum(K.square(tensor[0] - tensor[1]), axis=1, keepdims=True))

# Add the distance function to the network
distance1 = L1_layer([out1, out2])
distance2 = L1_layer([out2, out3])
print("distance1", distance1)

# distance1 - distance2
subtracted = keras.layers.Subtract()([distance1, distance2])
print("subrtacted", subtracted)


prediction = Dense(1, activation='sigmoid')(subtracted)
# prediction = Activation(K.sigmoid)(subtracted)

# -----------------------------------------------------

# print the model summary
model = Model(inputs=[input_points1, input_points2, input_points3], outputs=prediction)
print(model.summary())

## Visualize network
try:
  from keras.utils.vis_utils import plot_model as plot
  plot(model, to_file = 'drive/My Drive/Triplet_Network_o.png')
except ImportError:
  print('It seems like the dependencies for drawing the model (pydot, graphviz) are not installed')

In [ ]:
# Check if there is duplicates
print(df_results.duplicated().value_counts())
print(df_results_test.duplicated().value_counts())
df_concat = pd.concat([df_results_test, df_results])
print(df_concat.duplicated().value_counts())

False    5209
dtype: int64
False    264
dtype: int64
False    5473
dtype: int64


In [ ]:
# # Load preprocessed tada

# # Training data
# data_name = "_dining_train_3000"

# name1 = "drive/My Drive/Style_input_data/input_triplet_data1" + data_name + ".npy"
# input_data1_train = np.load(name1)
# name2 = "drive/My Drive/Style_input_data/input_triplet_data2" + data_name + ".npy"
# input_data2_train = np.load(name2)
# name3 = "drive/My Drive/Style_input_data/input_triplet_data3" + data_name + ".npy"
# input_data3_train = np.load(name3)
# name_y = "drive/My Drive/Style_input_data/y_data" + data_name
# y_data_train = pd.read_pickle(name_y)


# # # # Testing data
# data_name = "_dining_test"

# name1 = "drive/My Drive/Style_input_data_1/input_triplet_data1" + data_name + ".npy"
# input_data1_test = np.load(name1)
# name2 = "drive/My Drive/Style_input_data_1/input_triplet_data2" + data_name + ".npy"
# input_data2_test = np.load(name2)
# name3 = "drive/My Drive/Style_input_data_1/input_triplet_data2" + data_name + ".npy"
# input_data3_test = np.load(name3)
# name_y = "drive/My Drive/Style_input_data_1/y_data" + data_name 
# y_data_test = pd.read_pickle(name_y)


## Style Data 2 ##
### Training data
data_name = "_furniture_train_fold0"

name1 = "drive/My Drive/Style_input_data_2/input_triplet_data1" + data_name + ".npy"
input_data1_train = np.load(name1)
name2 = "drive/My Drive/Style_input_data_2/input_triplet_data2" + data_name + ".npy"
input_data2_train = np.load(name2)
name3 = "drive/My Drive/Style_input_data_2/input_triplet_data3" + data_name + ".npy"
input_data3_train = np.load(name3)
name_y = "drive/My Drive/Style_input_data_2/y_data" + data_name
y_data_train = pd.read_pickle(name_y)

### Testing data
data_name = "_furniture_test_fold0"

name1 = "drive/My Drive/Style_input_data_2/input_triplet_data1" + data_name + ".npy"
input_data1_test = np.load(name1)
name2 = "drive/My Drive/Style_input_data_2/input_triplet_data2" + data_name + ".npy"
input_data2_test = np.load(name2)
name3 = "drive/My Drive/Style_input_data_2/input_triplet_data3" + data_name + ".npy"
input_data3_test = np.load(name3)
name_y = "drive/My Drive/Style_input_data_2/y_data" + data_name
y_data_test = pd.read_pickle(name_y)

In [ ]:
y_data_train.shape

(16274, 4)

## Train on Style Data 1

In [ ]:
import keras.callbacks

# compile classification model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

####### Use for style data 1 #######
# Training and testing data
X_train = [input_data1_train,input_data2_train, input_data3_train]
Y_train = y_data_train["target"]

X_test = [input_data1_test,input_data2_test, input_data3_test]
Y_test = y_data_test["target"]



# Time
start = time.time()

# TensorBoard
tb_cb = keras.callbacks.TensorBoard(log_dir="drive/My Drive/tflog/")
cbks = [tb_cb]

num_epoch = 5
# Fit model on training data
for i in range(1, num_epoch):
    # rotate and jitter the points
#     train_points_rotate = rotate_point_cloud(train_points_r)
#     train_points_jitter = jitter_point_cloud(train_points_rotate)
    print("Total epoch:", i, "/", num_epoch)
    model.fit(X_train, Y_train, batch_size=16, epochs=1, shuffle=True, verbose=1, callbacks=cbks)
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time/60) + "[minutes]")
   
    model.save_weights("drive/My Drive/Weights/style_model_weights_living.h5")
    
    if i % 1 == 0:
      scores = model.evaluate(X_test, Y_test, verbose=1)
      print('Test loss: ', scores[0])
      print('Test accuracy: ', scores[1])


## Save model and weights
model.save_weights("drive/My Drive/Weights/style_model_weights_living.h5")

In [ ]:
y_data_train

## Train on Style Data 2 (cross validation)

In [ ]:
import keras.callbacks

# compile classification model
model.reset_states()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


####### Use for style data 2 #######
# Training and testing data
# The triplet is in the order ["a", "query", "b"]
X_train = [input_data1_train,input_data2_train, input_data3_train]
Y_train = y_data_train["target"]


X_test = [input_data1_test,input_data2_test, input_data3_test]
Y_test = y_data_test["target"]


# Time
start = time.time()

# TensorBoard
tb_cb = keras.callbacks.TensorBoard(log_dir="drive/My Drive/tflog/")
cbks = [tb_cb]

num_epoch = 10
# Fit model on training data
for i in range(1, num_epoch):
    # rotate and jitter the points
#     train_points_rotate = rotate_point_cloud(train_points_r)
#     train_points_jitter = jitter_point_cloud(train_points_rotate)
    print("Total epoch:", i, "/", num_epoch)
    model.fit(X_train, Y_train, batch_size=16, epochs=1, shuffle=True, verbose=1, callbacks=cbks)
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time/60) + "[minutes]")
   
    model.save_weights("drive/My Drive/Weights/style_model_weights.h5")
    
    if i % 1 == 0:
      scores = model.evaluate(X_test, Y_test, verbose=1)
      print('Test loss: ', scores[0])
      print('Test accuracy: ', scores[1])


## Save model and weights
model.save_weights("drive/My Drive/Weights/style_model_weights.h5")

In [ ]:
print(len(input_data3))
print(len(input_data3_train))
print(len(input_data3_test))

In [ ]:
print(len(Y))
print(len(Y_train))
print(len(Y_test))

In [ ]:
# Evaluate model on test data
Y_test = np_utils.to_categorical(y_data_test["target"], k)
X_test = [input_data1_test,input_data2_test, input_data3_test]
scores = model.evaluate(X_test, Y_test, verbose=1)
scores

In [ ]:
Y_train

In [ ]:
pred = model.predict(X_test,1,verbose=1)
pred

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 " hours"}'

In [ ]:
# Scaling
from sklearn import preprocessing
mm = preprocessing.MinMaxScaler()
for i in range(len(test_points)):
    test_points[i] = mm.fit_transform(test_points[i])